<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---
**Source:**<br>
*Dataset adapted from: Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning Algorithms. Han Xiao, Kashif Rasul, Roland Vollgraf. arXiv:1708.07747*


# Fashion - MNIST Image Classification Competition Model Submission Guide

---

Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data / Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [2]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/fashion_mnist_competition_data-repository:latest') 


Data downloaded successfully.


##2.   Preprocess data / Write and Save Preprocessor function


In [3]:
# Here is a pre-designed preprocessor, but you could also build your own to prepare the data differently

def preprocessor(image_filepath, shape=(28, 28)):
        """
        This function reads in images, resizes them to a fixed shape and
        min/max transforms them before converting feature values to float32 numeric values
        required by onnx files.
        
        params:
            image_filepath
                full filepath of a particular image
                      
        returns:
            X
                numpy array of preprocessed image data
        """
           
        import cv2
        import numpy as np

        "Resize a color image and min/max transform the image"
        img = cv2.imread(image_filepath) # Read in image from filepath.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of blue green and red, we reverse the order for ML.
        img = cv2.resize(img, shape) # Change height and width of image.
        img = img / 255.0 # Min-max transform.

        # Resize all the images...
        X = np.array(img)
        X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels] for keras model.
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        return X

In [4]:
#  Create training data objects

# Preprocess X_train image data to generate predictions from models 
import numpy as np

file_names = [('fashion_mnist_competition_data/training_data/train_' + str(i) + '.jpeg') for i in range(60000)]
preprocessed_image_data = [preprocessor(x) for x in file_names]

#Create single X_test array from preprocessed images
X_train = np.vstack(preprocessed_image_data) 

# Load y_train labels 
import pickle
with open("fashion_mnist_competition_data/y_train_labels.pkl", "rb") as fp:  
    y_train_labels = pickle.load(fp)

# One-hot encode y_train labels (y_train.columns used to generate prediction labels below)
import pandas as pd
y_train = pd.get_dummies(y_train_labels)

In [5]:
# Create test data objects

# Preprocess X_test image data to generate predictions from models 
import numpy as np

file_names = [('fashion_mnist_competition_data/test_data/test_' + str(i) + '.jpeg') for i in range(10000)]
preprocessed_image_data = [preprocessor(x) for x in file_names]

#Create single X_test array from preprocessed images
X_test = np.vstack(preprocessed_image_data) 

In [6]:
# Check shape 

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(60000, 28, 28, 3)
(10000, 28, 28, 3)
(60000, 10)


##3. Fit model on preprocessed data and save preprocessor function and model 


In [7]:
# Let's build a convnet model...
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers

with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
                                 
        cnn1 = Sequential()
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=[28, 28, 3]))
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        cnn1.add(MaxPooling2D(pool_size=2)) 
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        cnn1.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
        cnn1.add(MaxPooling2D(pool_size=2))
        cnn1.add(Dropout(0.3))
        cnn1.add(Flatten())
        cnn1.add(Dense(50, activation='relu'))
        cnn1.add(Dropout(0.2))
        cnn1.add(Dense(10, activation='softmax'))

        # Compile model...
        cnn1.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

        # Fitting the NN to the Training set...
        hist = cnn1.fit(X_train, y_train, epochs=1, verbose=1, validation_split=0.2)

1500/1500 [==============================] - 102s 66ms/step - loss: 0.6677 - accuracy: 0.7915 - val_loss: 0.4382 - val_accuracy: 0.8594


#### Save preprocessor function to local "preprocessor.zip" file

In [8]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [9]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(cnn1, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [10]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://vaw5a558u9.execute-api.us-east-1.amazonaws.com/prod/m" #This is the unique rest api that powers this Fashion-MNIST Classification Model Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [11]:
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

In [12]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=cnn1.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 3

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1662


In [13]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dropout_layers,dense_layers,flatten_layers,conv2d_layers,maxpooling2d_layers,relu_act,softmax_act,loss,optimizer,memory_size,username,version
0,84.84%,84.66%,84.95%,84.84%,keras,False,True,Sequential,11,47168,2.000000,2,1,4.000000,2.000000,5,1.000000,str,RMSprop,2697256,AIModelShare,3
1,84.56%,84.48%,85.61%,84.56%,keras,False,True,Sequential,3,302474,nan,2,1,nan,nan,1,nan,CategoricalCrossentropy,Adam,165512,AIModelShare,1
2,81.84%,81.88%,84.27%,81.84%,keras,False,True,Sequential,4,506186,nan,3,1,nan,nan,2,1.000000,function,RMSprop,179352,AIModelShare,2


## 5. Repeat submission process to improve place on leaderboard


In [14]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).

with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
      densemodel = Sequential()
      densemodel.add(Dense(128,  input_shape=(28, 28, 3), activation='relu'))
      densemodel.add(Dense(64, activation='relu'))
      densemodel.add(Dense(64, activation='relu'))
      densemodel.add(Flatten())
      densemodel.add(Dense(10, activation='softmax')) 
                                                  
      # Compile model
      densemodel.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

      # Fitting the NN to the Training set
      densemodel.fit(X_train, y_train, 
                    epochs = 1, verbose=1, validation_split=.2)

1500/1500 [==============================] - 134s 89ms/step - loss: 0.4940 - accuracy: 0.8212 - val_loss: 0.3979 - val_accuracy: 0.8602


In [15]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(densemodel, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [16]:
#Submit Model 2: 

#-- Generate predicted y values (Model 2)
prediction_column_index=densemodel.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 4

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1662


In [17]:
# Compare two or more models 
data=mycompetition.compare_models([3,4], verbose=1)
mycompetition.stylize_compare(data)

,Model_3_Layer,Model_3_Shape,Model_3_Params,Model_4_Layer,Model_4_Shape,Model_4_Params
0,Conv2D,"[None, 28, 28, 16]",448,Dense,"[None, 28, 28, 128]",512.000000
1,Conv2D,"[None, 28, 28, 16]",2320,Dense,"[None, 28, 28, 64]",8256.000000
2,MaxPooling2D,"[None, 14, 14, 16]",0,Dense,"[None, 28, 28, 64]",4160.000000
3,Conv2D,"[None, 14, 14, 16]",2320,Flatten,"[None, 50176]",0.000000
4,Conv2D,"[None, 14, 14, 16]",2320,Dense,"[None, 10]",501770.000000
5,MaxPooling2D,"[None, 7, 7, 16]",0,None,None,nan
6,Dropout,"[None, 7, 7, 16]",0,None,None,nan
7,Flatten,"[None, 784]",0,None,None,nan
8,Dense,"[None, 50]",39250,None,None,nan
9,Dropout,"[None, 50]",0,None,None,nan


## Optional: Tune model within range of hyperparameters with Keras Tuner

*Simple example shown below. Consult [documentation](https://keras.io/guides/keras_tuner/getting_started/) to see full functionality.*

In [ ]:
! pip install keras_tuner

In [19]:
#Separate validation data 
from sklearn.model_selection import train_test_split
x_train_split, x_val, y_train_split, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42)

In [20]:
import keras_tuner as kt
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers

#Define model structure & parameter search space with function
def build_model(hp):
    with tf.device('/device:GPU:0'): # "/GPU:0": Short-hand notation for the first GPU of your machine that is visible to TensorFlow.
                                 
        model = Sequential()
        model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=[28, 28, 3]))
        model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        model.add(MaxPooling2D(pool_size=2)) 
        model.add(Conv2D(filters=16, kernel_size=3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
        model.add(Conv2D(filters=hp.Int("filters", min_value=8, max_value=32, step=8), #range 8-32 inclusive, minimum step between tested values is 8
                         kernel_size=3, padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Dropout(0.3))
        model.add(Flatten())
        model.add(Dense(50, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(10, activation='softmax'))

        # Compile model...
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

        return model 

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="val_accuracy", # objective to optimize
    max_trials=2, #max number of trials to run during search
    executions_per_trial=1, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(x_train_split, y_train_split, epochs=1, validation_data=(x_val, y_val))


Trial 2 Complete [00h 02m 23s]
val_accuracy: 0.8683333396911621

Best val_accuracy So Far: 0.8683333396911621
Total elapsed time: 00h 04m 47s


In [21]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_model = build_model(best_hps[0])
# Fit with the entire dataset.
tuned_model.fit(x=X_train, y=y_train, epochs=1)


1875/1875 [==============================] - 124s 66ms/step - loss: 0.6359 - accuracy: 0.7983


In [22]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(tuned_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("tuned_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [23]:
#Submit Model 3: 

#-- Generate predicted y values (Model 3)
prediction_column_index=tuned_model.predict(X_test).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 3 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "tuned_model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 5

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1662


In [24]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dropout_layers,dense_layers,flatten_layers,conv2d_layers,maxpooling2d_layers,relu_act,softmax_act,loss,optimizer,memory_size,username,version
0,86.64%,86.55%,86.61%,86.64%,keras,False,True,Sequential,11,47168,2.000000,2,1,4.000000,2.000000,5,1.000000,str,RMSprop,2366416,AIModelShare,5
1,85.08%,84.84%,85.27%,85.08%,keras,False,True,Sequential,5,514698,nan,4,1,nan,nan,3,1.000000,str,RMSprop,2045824,AIModelShare,4
2,84.84%,84.66%,84.95%,84.84%,keras,False,True,Sequential,11,47168,2.000000,2,1,4.000000,2.000000,5,1.000000,str,RMSprop,2697256,AIModelShare,3
3,84.56%,84.48%,85.61%,84.56%,keras,False,True,Sequential,3,302474,nan,2,1,nan,nan,1,nan,CategoricalCrossentropy,Adam,165512,AIModelShare,1
4,81.84%,81.88%,84.27%,81.84%,keras,False,True,Sequential,4,506186,nan,3,1,nan,nan,2,1.000000,function,RMSprop,179352,AIModelShare,2


In [25]:
# Compare two or more models 
data=mycompetition.compare_models([4, 5], verbose=1)
mycompetition.stylize_compare(data)

,Model_4_Layer,Model_4_Shape,Model_4_Params,Model_5_Layer,Model_5_Shape,Model_5_Params
0,Dense,"[None, 28, 28, 128]",512.000000,Conv2D,"[None, 28, 28, 16]",448
1,Dense,"[None, 28, 28, 64]",8256.000000,Conv2D,"[None, 28, 28, 16]",2320
2,Dense,"[None, 28, 28, 64]",4160.000000,MaxPooling2D,"[None, 14, 14, 16]",0
3,Flatten,"[None, 50176]",0.000000,Conv2D,"[None, 14, 14, 16]",2320
4,Dense,"[None, 10]",501770.000000,Conv2D,"[None, 14, 14, 16]",2320
5,None,None,nan,MaxPooling2D,"[None, 7, 7, 16]",0
6,None,None,nan,Dropout,"[None, 7, 7, 16]",0
7,None,None,nan,Flatten,"[None, 784]",0
8,None,None,nan,Dense,"[None, 50]",39250
9,None,None,nan,Dropout,"[None, 50]",0
